<a href="https://colab.research.google.com/github/joaoeliandro/etl-cotacao-bovespa/blob/main/ETL_Bovespa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [7]:
def read_file(path, filename, year_date, type_file):
  _file = f'{path}{filename}{year_date}.{type_file}' 

  # Mapear colunas
  colspecs = [(2,10),
              (10,12),
              (12,24),
              (27,39),
              (56,69),
              (69,82),
              (82,95),
              (108,121),
              (152,170),
              (170,188)
  ]

  names = ['data_pregao', 
          'cod_bdi', 
          'sigla_acao', 
          'nome_acao', 
          'preco_abertura', 
          'preco_max', 
          'preco_min',
          'preco_fechamento',
          'qtd_negocios',
          'volume_negocios']

  df = pd.read_fwf(_file, colspecs=colspecs, names=names, skiprows=1)
  
  return df

In [8]:
# Filtrar ações
def filter_stocks(df):
  df = df [df['cod_bdi']==2]
  df = df.drop(['cod_bdi'], 1)
  
  return df

In [9]:
# Ajuste campo de data
def parse_date(df):
  df['data_pregao'] = pd.to_datetime(df['data_pregao'], format='%Y%m%d')
  
  return df

In [10]:
# Ajuste dos campos numéricos
def parse_values(df):
  df['preco_abertura'] = (df['preco_abertura']/100).astype(float)
  df['preco_max'] = (df['preco_max']/100).astype(float)
  df['preco_min'] = (df['preco_min']/100).astype(float)
  df['preco_fechamento'] = (df['preco_fechamento']/100).astype(float)
  
  return df

In [12]:
# Junção dos arquivos
def concat_files(path, filename, year_date, type_file, final_file):
  for i, y in enumerate(year_date):
    df = read_file(path, filename, y, type_file)
    df = filter_stocks(df)
    df = parse_date(df)
    df = parse_values(df)

    if i==0:
      df_final = df
    else:
      df_final = pd.concat([df_final, df])
  
  df_final.to_csv(f'{path}//{final_file}', index=False)

In [14]:
# Execução do script de ETL

year_date=['2021']

path=f'/content/'

filename='COTAHIST_A'

type_file='TXT'

final_file='all_bovespa.csv'

concat_files(path, filename, year_date, type_file, final_file)

<ipython-input-8-4fdc8973cf13>:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop(['cod_bdi'], 1)
